In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('tmpwyuGTwswimming.csv')

In [3]:
data

,Name,Address,Suburb,Postcode,State,Business Category,LGA,Region
0,Aquabear AUSSI Masters,La Trobe University Sports Centre,Bundoora,3083,VIC,Swimming Club,"Whittlesea,Darebin,Banyule",North and West Metropolitan Region
1,Bairnsdale Amateur Swimming Club,NaN,Bairnsdale,3875,VIC,Swimming Club,East Gippsland,Gippsland
2,Ballarat GCO Swim Club,PO Box 1709,Ballarat,3354,VIC,Swimming Club,Ballarat,Grampians
3,Broadmeadows Swimming Clubs,PO Box 605,Glenroy,3046,VIC,Swimming Club,Moreland,North and West Metropolitan Region
4,Bundoora Torpedoes Swim Club,NaN,Bundoora,3083,VIC,Swimming Club,Whittlesea ; Darebin ; Banyule,North and West Metropolitan Region
...,...,...,...,...,...,...,...,...
248,Splashdown,Coppards Road,Whittington,3219,VIC,"Personal training,Strength Training,Swimming Pool",Greater Geelong (C),Barwon South Western Region
249,Waterworld,Cnr Melbourne and Cox Roads,Norlane,3214,VIC,"Personal training,Strength Training,Swimming Pool",Greater Geelong (C),Barwon South Western Region
250,Life Activities Club Heidelberg,PO Box 245,Rosanna,3084,VIC,"Recreation Group,Swimming Club,Table Tennis Cl...",Banyule (C),North and West Metropolitan Region
251,Ballarat YMCA Community Base,183 Albert Street,Sebastopol,3356,VIC,"Recreation Group,Swimming Pool",Ballarat,Grampians


## From first record we can see that park_name `Aquabear AUSSI Masters` is classified in 3 LGA: `Whittlesea,Darebin,Banyule`.

## Let's validate this from the 'locality.csv'

In [4]:
locality = pd.read_csv('locality.csv')

In [5]:
locality

,LOCALITY_ID,SUBURB,LGA,POST_CODE
0,1000001,Abbeyard,ALPINE,3737
1,1000002,Abbotsford,YARRA,3067
2,1000003,Aberfeldie,MOONEE VALLEY,3040
3,1000004,Aberfeldy,BAW BAW,3825
4,1000005,Acheron,MURRINDINDI,3714
...,...,...,...,...
3216,1003217,Yulecart,SOUTHERN GRAMPIANS,3301
3217,1003218,Yundool,MOIRA,3727
3218,1003219,Yuroke,HUME,3063
3219,1003220,Yuulong,COLAC OTWAY,3237


In [6]:
locality.loc[(locality['SUBURB'] == 'Bundoora') & (locality['LGA'] == 'Whittlesea'.upper())]

,LOCALITY_ID,SUBURB,LGA,POST_CODE
426,1000427,Bundoora,WHITTLESEA,3083


In [7]:
locality.loc[(locality['SUBURB'] == 'Bundoora') & (locality['LGA'] == 'Darebin'.upper())]

,LOCALITY_ID,SUBURB,LGA,POST_CODE
425,1000426,Bundoora,DAREBIN,3083


In [8]:
locality.loc[(locality['SUBURB'] == 'Bundoora') & (locality['LGA'] == 'Banyule'.upper())]

,LOCALITY_ID,SUBURB,LGA,POST_CODE
424,1000425,Bundoora,BANYULE,3083


## Indeed, Suburb 'Bundoora' is part of all the 3 LGA (Whittlesea,Darebin,Banyule)

## For our database, we need this to be reflected in a long format. Hence we need to break down the LGA and make one record for each of the LGA 

## In order to achieve this, we need to iterate through each row of the dataframe and make records accordingly

### All the Swimming clubs are not swimming pools. So we remove this business type from data

In [9]:
data = data[data['Business Category'] != 'Swimming Club']

In [10]:
selected = ['Name', 'Suburb', 'LGA', 'Address']
selected_df = data[selected]
selected_df.reset_index(drop=True,inplace=True)

In [11]:
# check for missing data
for idx, row in selected_df.iterrows():
    if row.isnull().values.any():
        print(idx, row)

30 Name       Chewton Swimming Pool
Suburb                   Chewton
LGA                          NaN
Address    Chewton Memorial Park
Name: 30, dtype: object
34 Name       Coburg Outdoor Pool
Suburb                  Coburg
LGA                   Moreland
Address                    NaN
Name: 34, dtype: object
57 Name       Inglewood Swimming Pool
Suburb                   Inglewood
LGA                         Loddon
Address                        NaN
Name: 57, dtype: object
75 Name       Lismore District War Memorial Swimming Pool
Suburb                                         Lismore
LGA                                        Corangamite
Address                                            NaN
Name: 75, dtype: object
79 Name        Mansfield Outdoor Pool
Suburb                   Mansfield
LGA        Yarra Ranges , Delatite
Address                        NaN
Name: 79, dtype: object
82 Name       Mirboo North Swimming Pool
Suburb                   Mirboo North
LGA                   South Gi

## There are some records that don't have LGA but Suburb. So let's check what LGA they fall into with locality.csv

In [12]:
locality.loc[locality['SUBURB'] == 'Knox']

,LOCALITY_ID,SUBURB,LGA,POST_CODE


In [13]:
locality.loc[locality['SUBURB'] == 'Chewton']

,LOCALITY_ID,SUBURB,LGA,POST_CODE
586,1000587,Chewton,MOUNT ALEXANDER,3451


In [14]:
locality.loc[locality['SUBURB'] == 'Nyah']

,LOCALITY_ID,SUBURB,LGA,POST_CODE
2176,1002177,Nyah,SWAN HILL RURAL,3594


In [15]:
locality.loc[locality['SUBURB'] == 'Tallarook']

,LOCALITY_ID,SUBURB,LGA,POST_CODE
2666,1002667,Tallarook,MITCHELL,3659


In [16]:
locality.loc[locality['SUBURB'] == 'Tallarook']['LGA'].values[0]

'MITCHELL'

## After research, 'Knox' is actually an LGA but not a suburb. So I believe this is a misstake by entry. Hence I will manually change it to KNOX LGA

## Iterate through dataframe and change LGA accordingly

In [17]:
# iterrows doesn't change values from origional row unless you change values on df.copy() 
for idx, row in selected_df.iterrows():
    selected_df = selected_df.copy()
    if type(row['LGA']) == float:
        if row['Suburb'] in locality['SUBURB'].values:
            
            # if we can find this row's 'Suburb' in locality's 'SUBURB', then
            # change this row's 'LGA' to locality look up value based on the row's 'Suburb' value
            selected_df.at[idx, 'LGA'] = locality.loc[locality['SUBURB'] == row['Suburb']]['LGA'].values[0]
        
        else:
            selected_df.at[idx, 'LGA'] = row['Suburb']


In [18]:
# check for missing data
for idx, row in selected_df.iterrows():
    if row.isnull().values.any():
        print(idx, row)

34 Name       Coburg Outdoor Pool
Suburb                  Coburg
LGA                   Moreland
Address                    NaN
Name: 34, dtype: object
57 Name       Inglewood Swimming Pool
Suburb                   Inglewood
LGA                         Loddon
Address                        NaN
Name: 57, dtype: object
75 Name       Lismore District War Memorial Swimming Pool
Suburb                                         Lismore
LGA                                        Corangamite
Address                                            NaN
Name: 75, dtype: object
79 Name        Mansfield Outdoor Pool
Suburb                   Mansfield
LGA        Yarra Ranges , Delatite
Address                        NaN
Name: 79, dtype: object
82 Name       Mirboo North Swimming Pool
Suburb                   Mirboo North
LGA                   South Gippsland
Address                           NaN
Name: 82, dtype: object
87 Name       Nhill Swimming Baths
Suburb                    Nhill
LGA                   

## After dealing with Missing values or wrong values, now we can start to fix the Multiple LGA problem

In [19]:
selected_df.head(25)

,Name,Suburb,LGA,Address
0,Benalla Aquatic Centre,Benalla,Delatite,4 Mair Street
1,Diamond Valley Swimming Club,Greensbourgh,Whittlesea; Darebin; Banyule,Flintoff Street
2,Northern Swimming Pool,Balwyn North,Boroondara,271 Belmore Road
3,Swimland,Glen Waverley,Monash,452 Springvale Road
4,Warburton Outdoor Pool,Warburton,Yarra Ranges,Warburton Highway
5,Apollo Bay Swimming Pool,Apollo Bay,Colac-Otway (S),Costin Street
6,Aquacoaster Water Slide,Mildura,Mildura Rural,18 Orange Avenue
7,Aquastyle,Mill Park,Whittlesea,5 Mill Park Drive
8,Aquastyle,Mill Park,Whittlesea,5 Development Boulevard
9,Ararat Solar Olympic Pool,Ararat,Ararat (RC),Alexander Gardens


In [20]:
counter = 0
for idx, row in selected_df.iterrows():
    if not row['LGA'].isalpha():
        counter+=1
        
print(counter)

148


In [21]:
multi_lga_list = []
for idx, row in selected_df.iterrows():
    if not row['LGA'].isalpha():
        multi_lga_list.append(row['LGA'])
multi_lga_list

['Whittlesea; Darebin; Banyule',
 'Yarra Ranges',
 'Colac-Otway (S)',
 'Mildura Rural',
 'Ararat (RC)',
 'Mornington Peninsula',
 'Wyndham , Moorabool , Melton',
 'East Gippsland',
 'Moorabool (S)',
 'Yarra Ranges',
 'Greater Bendigo',
 'Greater Bendigo',
 'Greater Bendigo (C)',
 'Greater Bendigo',
 'Nillumbik,Banyule',
 'Boroondara (C)',
 'Glen Eira',
 'MOUNT ALEXANDER',
 'Mornington Peninsula',
 'Maroondah (C)',
 'Greater Dandenong , Casey',
 'Central Goldfields',
 'Glen Eira',
 'Greater Bendigo',
 'South Gippsland',
 'Macedon Ranges , Melton',
 'Greater Bendigo',
 'Nillumbik,Banyule',
 'Southern Grampians',
 'Stonnington (C); Boroondara (C)',
 'Yarra Ranges',
 'Yarra Ranges',
 'Greater Bendigo',
 'Mildura Rural',
 'Greater Bendigo',
 'Greater Geelong',
 'Yarra Ranges , Knox',
 'Cardinia , Casey',
 'South Gippsland',
 'Greater Shepparton , Campaspe',
 'Macedon Ranges',
 'Macedon Ranges',
 'Greater Geelong (C)',
 'Greater Geelong',
 'Gannawarra , Campaspe',
 'South Gippsland',
 'Yarra

In [22]:
import re
pattern = re.compile(r'\s\W\w+\W+') # pattern for ' (C); ' or ' (C)' or ' (RC)'

In [23]:
re.sub(pattern, ',', 'Ararat')

'Ararat'

In [24]:
for idx, row in selected_df.iterrows():
    selected_df = selected_df.copy()
    selected_df.at[idx, 'LGA'] = re.sub(pattern, ',', row['LGA'])

In [25]:
selected_df.iloc[53]

Name       Healesville Community Indoor Heated Pool
Suburb                                  Healesville
LGA                                    Yarra Ranges
Address                               Camerons Road
Name: 53, dtype: object

In [26]:
selected_df.iloc[100]

Name       Pyramid Hill Swimming Pool
Suburb                   Pyramid Hill
LGA                            Loddon
Address               Victoria Street
Name: 100, dtype: object

In [27]:
pattern = re.compile(r'\s\W\s') # pattern for ' , ' or ' ; '

In [28]:
for idx, row in selected_df.iterrows():
    selected_df = selected_df.copy()
    selected_df.at[idx, 'LGA'] = re.sub(pattern, ',', row['LGA'])

In [29]:
selected_df.head(25)

,Name,Suburb,LGA,Address
0,Benalla Aquatic Centre,Benalla,Delatite,4 Mair Street
1,Diamond Valley Swimming Club,Greensbourgh,Whittlesea; Darebin; Banyule,Flintoff Street
2,Northern Swimming Pool,Balwyn North,Boroondara,271 Belmore Road
3,Swimland,Glen Waverley,Monash,452 Springvale Road
4,Warburton Outdoor Pool,Warburton,Yarra Ranges,Warburton Highway
5,Apollo Bay Swimming Pool,Apollo Bay,"Colac-Otway,",Costin Street
6,Aquacoaster Water Slide,Mildura,Mildura Rural,18 Orange Avenue
7,Aquastyle,Mill Park,Whittlesea,5 Mill Park Drive
8,Aquastyle,Mill Park,Whittlesea,5 Development Boulevard
9,Ararat Solar Olympic Pool,Ararat,"Ararat,",Alexander Gardens


In [30]:
pattern = re.compile(r'\W\B') # pattern for last special character like 'Knox,'

In [31]:
for idx, row in selected_df.iterrows():
    selected_df = selected_df.copy()
    selected_df.at[idx, 'LGA'] = re.sub(pattern, '', row['LGA'])

## Now all the LGA are in this pattern: \w+,\w+
## Next thing is to split the multiple LGA rows and duplicate the records for each single LGA

In [32]:
# create a list to store idx and LGA pair
swim_lga_list = []

for idx, row in selected_df.iterrows():
    if not row['LGA'].isalpha():
        LGA_list = row['LGA'].split(',')
        temp = []
        temp.append(idx)
        temp.append(LGA_list)
        
        swim_lga_list.append(temp)

In [33]:
swim_lga_list

[[1, ['Whittlesea Darebin Banyule']],
 [4, ['Yarra Ranges']],
 [5, ['Colac-Otway']],
 [6, ['Mildura Rural']],
 [12, ['Mornington Peninsula']],
 [13, ['Wyndham', 'Moorabool', 'Melton']],
 [14, ['East Gippsland']],
 [17, ['Yarra Ranges']],
 [18, ['Greater Bendigo']],
 [19, ['Greater Bendigo']],
 [20, ['Greater Bendigo']],
 [23, ['Greater Bendigo']],
 [24, ['Nillumbik', 'Banyule']],
 [28, ['Glen Eira']],
 [30, ['MOUNT ALEXANDER']],
 [36, ['Mornington Peninsula']],
 [41, ['Greater Dandenong', 'Casey']],
 [42, ['Central Goldfields']],
 [43, ['Glen Eira']],
 [45, ['Greater Bendigo']],
 [47, ['South Gippsland']],
 [48, ['Macedon Ranges', 'Melton']],
 [49, ['Greater Bendigo']],
 [50, ['Nillumbik', 'Banyule']],
 [51, ['Southern Grampians']],
 [52, ['Stonnington', 'Boroondara']],
 [53, ['Yarra Ranges']],
 [54, ['Yarra Ranges']],
 [55, ['Greater Bendigo']],
 [58, ['Mildura Rural']],
 [60, ['Greater Bendigo']],
 [61, ['Greater Geelong']],
 [63, ['Yarra Ranges', 'Knox']],
 [64, ['Cardinia', 'Casey'

In [34]:
# false_pairs are something like [25, ['Port Phillip']] where the second element in the list returns False when tested with
# str.isalpha() but actually it is a one-element-list. The reason that caused this is that this element has a white space in
# its string
false_pairs = []
for pairs in swim_lga_list:
    if len(pairs[1]) == 1:
        false_pairs.append(pairs)

In [35]:
false_pairs

[[1, ['Whittlesea Darebin Banyule']],
 [4, ['Yarra Ranges']],
 [5, ['Colac-Otway']],
 [6, ['Mildura Rural']],
 [12, ['Mornington Peninsula']],
 [14, ['East Gippsland']],
 [17, ['Yarra Ranges']],
 [18, ['Greater Bendigo']],
 [19, ['Greater Bendigo']],
 [20, ['Greater Bendigo']],
 [23, ['Greater Bendigo']],
 [28, ['Glen Eira']],
 [30, ['MOUNT ALEXANDER']],
 [36, ['Mornington Peninsula']],
 [42, ['Central Goldfields']],
 [43, ['Glen Eira']],
 [45, ['Greater Bendigo']],
 [47, ['South Gippsland']],
 [49, ['Greater Bendigo']],
 [51, ['Southern Grampians']],
 [53, ['Yarra Ranges']],
 [54, ['Yarra Ranges']],
 [55, ['Greater Bendigo']],
 [58, ['Mildura Rural']],
 [60, ['Greater Bendigo']],
 [61, ['Greater Geelong']],
 [65, ['South Gippsland']],
 [67, ['Macedon Ranges']],
 [68, ['Macedon Ranges']],
 [69, ['Greater Geelong']],
 [70, ['Greater Geelong']],
 [73, ['South Gippsland']],
 [74, ['Yarra Ranges']],
 [77, ['Mount Alexander']],
 [78, ['Swan Hill Rural']],
 [80, ['Mildura Rural']],
 [82, ['S

## [49, ['Whittlesea Darebin Banyule']] 

## This record is actually not an LGA. It's actually 3 LGA but deliminated by single space

In [36]:
for pairs in false_pairs:
    
    swim_lga_list.remove(pairs)

In [37]:
swim_lga_list.append([1, 'Whittlesea Darebin Banyule'.split()])

In [38]:
swim_lga_list

[[13, ['Wyndham', 'Moorabool', 'Melton']],
 [24, ['Nillumbik', 'Banyule']],
 [41, ['Greater Dandenong', 'Casey']],
 [48, ['Macedon Ranges', 'Melton']],
 [50, ['Nillumbik', 'Banyule']],
 [52, ['Stonnington', 'Boroondara']],
 [63, ['Yarra Ranges', 'Knox']],
 [64, ['Cardinia', 'Casey']],
 [66, ['Greater Shepparton', 'Campaspe']],
 [72, ['Gannawarra', 'Campaspe']],
 [79, ['Yarra Ranges', 'Delatite']],
 [104, ['Manningham', 'Maroondah']],
 [107, ['Greater Shepparton', 'Campaspe']],
 [110, ['Murrindindi', 'Mitchell']],
 [111, ['Pyrenees', 'Corangamite']],
 [114, ['Whittlesea', 'Hume']],
 [125, ['Wyndham', 'Melton', 'Brimbank']],
 [132, ['Frankston', 'Casey']],
 [137, ['Nillumbik', 'Banyule']],
 [140, ['Kingston', 'Bayside']],
 [143, ['Yarra Ranges', 'Maroondah']],
 [152, ['Whitehorse', 'Boroondara']],
 [153, ['Moonee Valley', 'Maribyrnong']],
 [161, ['Maroondah', 'Knox']],
 [162, ['Moonee Valley', 'Maribyrnong']],
 [163, ['Wyndham', 'Melton']],
 [166, ['Melton', 'Hume']],
 [167, ['Maribyrnon

## Make a new dataframe to duplicate the records for each LGA

In [39]:
duplicate_LGA = pd.DataFrame(columns=['Name', 'Suburb', 'LGA', 'Address'])

In [40]:
name_list = []
suburb_list = []
lga_list = []
address_list = []
for pairs in swim_lga_list:
    for lga in pairs[1]:
        lga_list.append(lga)
        
        # store data in a temporary variable then append it to corresponding list 
        name = selected_df.iloc[pairs[0]]['Name']
        suburb = selected_df.iloc[pairs[0]]['Suburb']
        address = selected_df.iloc[pairs[0]]['Address']
        
        suburb_list.append(suburb)
        name_list.append(name)
        address_list.append(address)

In [41]:
duplicate_LGA['Name'] = name_list
duplicate_LGA['Suburb'] = suburb_list
duplicate_LGA['LGA'] = lga_list
duplicate_LGA['Address'] = address_list

In [42]:
duplicate_LGA

,Name,Suburb,LGA,Address
0,Bacchus Marsh Outdoor Pool,Bacchus Marsh,Wyndham,Grant Street
1,Bacchus Marsh Outdoor Pool,Bacchus Marsh,Moorabool,Grant Street
2,Bacchus Marsh Outdoor Pool,Bacchus Marsh,Melton,Grant Street
3,Briar Hill Swimming Pool Centre,Briar Hill,Nillumbik,"Shop 5, 111 Mountain View Road"
4,Briar Hill Swimming Pool Centre,Briar Hill,Banyule,"Shop 5, 111 Mountain View Road"
...,...,...,...,...
75,Vicki Field Swim School,Ringwood,Manningham,108 New Street
76,Vicki Field Swim School,Ringwood,Maroondah,108 New Street
77,Diamond Valley Swimming Club,Greensbourgh,Whittlesea,Flintoff Street
78,Diamond Valley Swimming Club,Greensbourgh,Darebin,Flintoff Street


In [43]:
for pairs in swim_lga_list:
    
    selected_df.drop(pairs[0],inplace=True)

In [44]:
selected_df

,Name,Suburb,LGA,Address
0,Benalla Aquatic Centre,Benalla,Delatite,4 Mair Street
2,Northern Swimming Pool,Balwyn North,Boroondara,271 Belmore Road
3,Swimland,Glen Waverley,Monash,452 Springvale Road
4,Warburton Outdoor Pool,Warburton,Yarra Ranges,Warburton Highway
5,Apollo Bay Swimming Pool,Apollo Bay,Colac-Otway,Costin Street
...,...,...,...,...
200,Splashdown,Whittington,Greater Geelong,Coppards Road
201,Waterworld,Norlane,Greater Geelong,Cnr Melbourne and Cox Roads
202,Life Activities Club Heidelberg,Rosanna,Banyule,PO Box 245
203,Ballarat YMCA Community Base,Sebastopol,Ballarat,183 Albert Street


In [45]:
swimming_pool = pd.concat([selected_df, duplicate_LGA])

In [46]:
swimming_pool

,Name,Suburb,LGA,Address
0,Benalla Aquatic Centre,Benalla,Delatite,4 Mair Street
2,Northern Swimming Pool,Balwyn North,Boroondara,271 Belmore Road
3,Swimland,Glen Waverley,Monash,452 Springvale Road
4,Warburton Outdoor Pool,Warburton,Yarra Ranges,Warburton Highway
5,Apollo Bay Swimming Pool,Apollo Bay,Colac-Otway,Costin Street
...,...,...,...,...
75,Vicki Field Swim School,Ringwood,Manningham,108 New Street
76,Vicki Field Swim School,Ringwood,Maroondah,108 New Street
77,Diamond Valley Swimming Club,Greensbourgh,Whittlesea,Flintoff Street
78,Diamond Valley Swimming Club,Greensbourgh,Darebin,Flintoff Street


In [47]:
swimming_pool['LGA'] = swimming_pool['LGA'].apply(lambda x: x.upper())

In [48]:
locality = pd.read_csv('locality.csv')

In [49]:
locality_lga = locality['LGA'].sort_values().unique()
swimming_lga = swimming_pool['LGA'].sort_values().unique()

In [50]:
locality_difference = set(swimming_lga) - set(locality_lga)

In [51]:
locality_difference

{'ARARAT',
 'COLAC-OTWAY',
 'DELATITE',
 'HORSHAM',
 'MORNINGTON PENINSULA',
 'WANGARATTA',
 'WODONGA RURAL'}

## CHEWTON is actually a suburb of MOUNT ALEXANDER (LGA), NYAH is a suburb of SWAN HILL RURAL, TALLAROOK is a suburb of MITCHELL

## ARARAT, COLAC-OTWAY, DELATITE, HORSHAM, LA TROBE, WANGARATTA, are LGA that locality doesn't have. But after examination, I found that these are either different naming convention or old LGA that now belong to another LGA

In [52]:
swimming_pool['LGA'].mask(swimming_pool['LGA'] == 'MORNINGTON PENINSULA', 'MORNINGTON', inplace = True)
swimming_pool['LGA'].mask(swimming_pool['LGA'] == 'CHEWTON', 'MOUNT ALEXANDER', inplace = True)
swimming_pool['LGA'].mask(swimming_pool['LGA'] == 'WODONGA RURAL', 'WODONGA', inplace = True)
swimming_pool['LGA'].mask(swimming_pool['LGA'] == 'NYAH', 'SWAN HILL RURAL', inplace = True)
swimming_pool['LGA'].mask(swimming_pool['LGA'] == 'TALLAROOK', 'MITCHELL', inplace = True)
swimming_pool['LGA'].mask(swimming_pool['LGA'] == 'ARARAT', 'ARARAT RURAL', inplace = True)
swimming_pool['LGA'].mask(swimming_pool['LGA'] == 'COLAC-OTWAY', 'COLAC OTWAY', inplace = True)
swimming_pool['LGA'].mask(swimming_pool['LGA'] == 'DELATITE', 'MANSFIELD', inplace = True)
swimming_pool['LGA'].mask(swimming_pool['LGA'] == 'HORSHAM', 'HORSHAM RURAL', inplace = True)
swimming_pool['LGA'].mask(swimming_pool['LGA'] == 'LA TROBE', 'LATROBE', inplace = True)
swimming_pool['LGA'].mask(swimming_pool['LGA'] == 'WANGARATTA', 'WANGARATTA RURAL', inplace = True)

In [53]:
final_sp = pd.DataFrame({'LGA': swimming_pool['LGA'],
                        'PARK_NAME' : swimming_pool['Name'],
                        'CATEGORY' : 'Swimming pool',
                        'ADDRESS' : swimming_pool['Address']})

In [54]:
final_sp.drop_duplicates(keep='first', inplace=True)

In [55]:
final_sp.to_csv('swimming pool final.csv', index=False)